In [24]:
import pandas as pd
import numpy as np

In [4]:
# Loading data
raw_data = pd.read_csv('Datasets_MS_Project/Landuse/LandUse_E_All_Data_(Normalized)/Inputs_LandUse_E_All_Data_(Normalized).csv')
raw_data.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf4 in position 146526: invalid continuation byte

In [25]:
# Loading data using latin1 encoder
raw_data = pd.read_csv('Datasets_MS_Project/Landuse/LandUse_E_All_Data_(Normalized)/Inputs_LandUse_E_All_Data_(Normalized).csv', encoding='latin1')
raw_data.head()

,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag,Note
0,2,'004,Afghanistan,6600,Country area,5110,Area,1961,1961,1000 ha,65286.0,A,NaN
1,2,'004,Afghanistan,6600,Country area,5110,Area,1962,1962,1000 ha,65286.0,A,NaN
2,2,'004,Afghanistan,6600,Country area,5110,Area,1963,1963,1000 ha,65286.0,A,NaN
3,2,'004,Afghanistan,6600,Country area,5110,Area,1964,1964,1000 ha,65286.0,A,NaN
4,2,'004,Afghanistan,6600,Country area,5110,Area,1965,1965,1000 ha,65286.0,A,NaN


In [26]:
raw_data.shape

(398774, 13)

In [27]:
raw_data['Item'].unique()

array(['Country area', 'Land area', 'Agriculture', 'Agricultural land',
       'Cropland', 'Arable land', 'Temporary crops',
       'Temporary meadows and pastures', 'Temporary fallow',
       'Permanent crops', 'Permanent meadows and pastures',
       'Perm. meadows & pastures - Nat. growing', 'Forest land',
       'Naturally regenerating forest', 'Planted Forest', 'Other land',
       'Inland waters', 'Land area equipped for irrigation',
       'Land area actually irrigated',
       'Agriculture area actually irrigated',
       'Agriculture area under organic agric.',
       'Agriculture area certified organic', 'Primary Forest',
       'Farm buildings and Farmyards', 'Coastal waters',
       'Cropland area actually irrigated',
       'Cropland area under organic agric.',
       'Cropland area certified organic',
       'Cropland area under protective cover',
       'Perm. meadows & pastures - Cultivated',
       'Land used for aquaculture',
       'Perm. meadows & pastures area actu

To predict farmer producer prices, not all land use indicators are directly 
relevant, but several can serve as useful predictive features, especially if:

- They indicate area available for production
- They relate to production methods (e.g., organic, tillage practices)
- They hint at input use intensity (e.g., irrigation)
- They capture land suitability or crop area

Following are the most relevant features: 
- Agricultural land
- Cropland
- Arable land
- Temporary crops
- Permanent crops
- Land area equipped for irrigation
- Land area actually irrigated
- Agriculture area actually irrigated
- Cropland area actually irrigated
- Cropland area under organic agric. 
- Cropland area under certified organic
- Cropland area under conventional tillage
- Cropland area under conservation tillage
- Cropland area under zero or no tillage

In [28]:
raw_data['Element'].unique()

array(['Area', 'Share in Land area',
       'Value of agricultural production (Int. $) per Area',
       'Share in Agricultural land', 'Area per capita',
       'Share in Forest land', 'Share in Cropland',
       'Carbon stock in living biomass'], dtype=object)

Following are the most relevant elements for our project: 
- **Area:** Gives absolute land size under a use. Affects total production supply and thus price.
- **Value of agricultural production (Int. $) per Area:** reflects productivity and economic value per hectare.
- **Area per capita:** Indicates land pressure, which can affect market prices

In [29]:
# filter raw data for desired items and elements
items_to_keep = [
    'Agricultural land',
    'Cropland',
    'Arable land',
    'Temporary crops',
    'Permanent crops',
    'Land area equipped for irrigation',
    'Land area actually irrigated',
    'Agriculture area actually irrigated',
    'Cropland area actually irrigated',
    'Cropland area under organic agric.', 
    'Cropland area under certified organic',
    'Cropland area under conventional tillage',
    'Cropland area under conservation tillage',
    'Cropland area under zero or no tillage'
]

elements_to_keep = [
    'Area',
    'Value of agricultural production (Int. $) per Area',
    'Area per capita'
]

filtered_data = raw_data.loc[
    (raw_data['Item'].isin(items_to_keep)) &
    (raw_data['Element'].isin(elements_to_keep))
]

filtered_data.head()

,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag,Note
186,2,'004,Afghanistan,6610,Agricultural land,5110,Area,1961,1961,1000 ha,37750.0,I,NaN
187,2,'004,Afghanistan,6610,Agricultural land,5110,Area,1962,1962,1000 ha,37800.0,I,NaN
188,2,'004,Afghanistan,6610,Agricultural land,5110,Area,1963,1963,1000 ha,37850.0,I,NaN
189,2,'004,Afghanistan,6610,Agricultural land,5110,Area,1964,1964,1000 ha,37905.0,I,NaN
190,2,'004,Afghanistan,6610,Agricultural land,5110,Area,1965,1965,1000 ha,37910.0,I,NaN


In [ ]:
# units for the measurements
filtered_data['Unit'].unique()

array(['1000 ha', 'USD_PPP/ha', 'ha/cap'], dtype=object)

## Area under different land use

In [30]:
# filter for desired data
filtered_data_1 = filtered_data.loc[filtered_data['Element']=='Area']
filtered_data_1.shape

(80923, 13)

In [17]:
filtered_data_1['Item'].value_counts()

Item
Agricultural land                           15353
Cropland                                    15160
Arable land                                 14974
Permanent crops                             14123
Land area equipped for irrigation           12426
Temporary crops                              5738
Agriculture area actually irrigated          1115
Cropland area under organic agric.            987
Cropland area actually irrigated              653
Land area actually irrigated                  202
Cropland area under conventional tillage       75
Cropland area under zero or no tillage         63
Cropland area under conservation tillage       54
Name: count, dtype: int64

There is a lots of missing data for several of the items here. Probably we would 
keep the top 5 items only.

In [33]:
# dropping items with fewer observations
items_to_keep = [
    'Agricultural land', 
    'Cropland', 
    'Arable land', 
    'Permanent crops', 
    'Land area equipped for irrigation', 
    'Temporary crops'
    ]

filtered_data_1 = filtered_data_1.loc[
    filtered_data_1['Item'].isin(items_to_keep)
]

filtered_data_1['Item'].unique()

array(['Agricultural land', 'Cropland', 'Arable land', 'Temporary crops',
       'Permanent crops', 'Land area equipped for irrigation'],
      dtype=object)

In [34]:
# restructuring dataset from long to wide format
pivoted_data_1 = filtered_data_1.pivot_table(
    index = ['Area Code', 'Area', 'Year Code', 'Year'],
    columns = 'Item',
    values = 'Value'
)

# resetting row index
pivoted_data_1.reset_index(inplace=True)

# setting name of column index to None
pivoted_data_1.columns.name = None

pivoted_data_1.head()

,Area Code,Area,Year Code,Year,Agricultural land,Arable land,Cropland,Land area equipped for irrigation,Permanent crops,Temporary crops
0,1,Armenia,1992,1992,1170.0,423.0,483.0,276.0,60.0,NaN
1,1,Armenia,1993,1993,1181.0,433.0,493.0,277.8,60.0,NaN
2,1,Armenia,1994,1994,1200.0,435.0,500.0,279.6,65.0,NaN
3,1,Armenia,1995,1995,1244.0,435.0,494.0,281.4,59.0,NaN
4,1,Armenia,1996,1996,1306.0,450.0,506.0,283.2,56.0,NaN


In [39]:
# renaming columns
cleaned_data_1 = pivoted_data_1.rename(
    columns = {
        'Area Code': 'area_code',
        'Area': 'area',
        'Year Code': 'year_code',
        'Year': 'year',
        'Agricultural land': 'area_agri_land', 
        'Cropland': 'area_cropland', 
        'Arable land': 'area_arable_land', 
        'Temporary crops': 'area_temporary_crops',
        'Permanent crops': 'area_permanent_crops', 
        'Land area equipped for irrigation': 'area_with_irrigation' 
    }
)

cleaned_data_1.head()

,area_code,area,year_code,year,area_agri_land,area_arable_land,area_cropland,area_with_irrigation,area_permanent_crops,area_temporary_crops
0,1,Armenia,1992,1992,1170.0,423.0,483.0,276.0,60.0,NaN
1,1,Armenia,1993,1993,1181.0,433.0,493.0,277.8,60.0,NaN
2,1,Armenia,1994,1994,1200.0,435.0,500.0,279.6,65.0,NaN
3,1,Armenia,1995,1995,1244.0,435.0,494.0,281.4,59.0,NaN
4,1,Armenia,1996,1996,1306.0,450.0,506.0,283.2,56.0,NaN


In [46]:
cleaned_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15354 entries, 0 to 15353
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   area_code             15354 non-null  int64  
 1   area                  15354 non-null  object 
 2   year_code             15354 non-null  int64  
 3   year                  15354 non-null  int64  
 4   area_agri_land        15353 non-null  float64
 5   area_arable_land      14974 non-null  float64
 6   area_cropland         15160 non-null  float64
 7   area_with_irrigation  12426 non-null  float64
 8   area_permanent_crops  14123 non-null  float64
 9   area_temporary_crops  5738 non-null   float64
dtypes: float64(6), int64(3), object(1)
memory usage: 1.2+ MB


## Value of agricultural production (Int. $) per Area

In [36]:
# filter for desired data
filtered_data_2 = filtered_data.loc[filtered_data['Element']=='Value of agricultural production (Int. $) per Area']
filtered_data_2.shape

(13665, 13)

In [37]:
filtered_data_2['Item'].value_counts()

Item
Agricultural land    13665
Name: count, dtype: int64

In [38]:
# restructuring dataset from long to wide format
pivoted_data_2 = filtered_data_2.pivot_table(
    index = ['Area Code', 'Area', 'Year Code', 'Year'],
    columns = 'Item',
    values = 'Value'
)

# resetting row index
pivoted_data_2.reset_index(inplace=True)

# setting name of column index to None
pivoted_data_2.columns.name = None

pivoted_data_2.head()

,Area Code,Area,Year Code,Year,Agricultural land
0,1,Armenia,1997,1997,637.93
1,1,Armenia,1998,1998,694.61
2,1,Armenia,1999,1999,691.05
3,1,Armenia,2000,2000,670.41
4,1,Armenia,2001,2001,693.99


In [40]:
# renaming columns
cleaned_data_2 = pivoted_data_2.rename(
    columns = {
        'Area Code': 'area_code',
        'Area': 'area',
        'Year Code': 'year_code',
        'Year': 'year',
        'Agricultural land': 'value_per_unit_agri_land'
    }
)

cleaned_data_2.head()

,area_code,area,year_code,year,value_per_unit_agri_land
0,1,Armenia,1997,1997,637.93
1,1,Armenia,1998,1998,694.61
2,1,Armenia,1999,1999,691.05
3,1,Armenia,2000,2000,670.41
4,1,Armenia,2001,2001,693.99


In [45]:
cleaned_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13665 entries, 0 to 13664
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   area_code                 13665 non-null  int64  
 1   area                      13665 non-null  object 
 2   year_code                 13665 non-null  int64  
 3   year                      13665 non-null  int64  
 4   value_per_unit_agri_land  13665 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 533.9+ KB


In [48]:
# merging two dataframes with left join
merged_data_1 = pd.merge(
    cleaned_data_1, cleaned_data_2,
    on = ['area_code', 'area', 'year_code', 'year'],
    how = 'left'
)
merged_data_1.head()

,area_code,area,year_code,year,area_agri_land,area_arable_land,area_cropland,area_with_irrigation,area_permanent_crops,area_temporary_crops,value_per_unit_agri_land
0,1,Armenia,1992,1992,1170.0,423.0,483.0,276.0,60.0,NaN,NaN
1,1,Armenia,1993,1993,1181.0,433.0,493.0,277.8,60.0,NaN,NaN
2,1,Armenia,1994,1994,1200.0,435.0,500.0,279.6,65.0,NaN,NaN
3,1,Armenia,1995,1995,1244.0,435.0,494.0,281.4,59.0,NaN,NaN
4,1,Armenia,1996,1996,1306.0,450.0,506.0,283.2,56.0,NaN,NaN


In [49]:
merged_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15354 entries, 0 to 15353
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   area_code                 15354 non-null  int64  
 1   area                      15354 non-null  object 
 2   year_code                 15354 non-null  int64  
 3   year                      15354 non-null  int64  
 4   area_agri_land            15353 non-null  float64
 5   area_arable_land          14974 non-null  float64
 6   area_cropland             15160 non-null  float64
 7   area_with_irrigation      12426 non-null  float64
 8   area_permanent_crops      14123 non-null  float64
 9   area_temporary_crops      5738 non-null   float64
 10  value_per_unit_agri_land  13665 non-null  float64
dtypes: float64(7), int64(3), object(1)
memory usage: 1.3+ MB


## Area per capita

In [41]:
# filter for desired data
filtered_data_3 = filtered_data.loc[filtered_data['Element']=='Area per capita']
filtered_data_3.shape

(15098, 13)

In [42]:
filtered_data_3['Item'].value_counts()

Item
Cropland    15098
Name: count, dtype: int64

In [43]:
# restructuring dataset from long to wide format
pivoted_data_3 = filtered_data_3.pivot_table(
    index = ['Area Code', 'Area', 'Year Code', 'Year'],
    columns = 'Item',
    values = 'Value'
)

# resetting row index
pivoted_data_3.reset_index(inplace=True)

# setting name of column index to None
pivoted_data_3.columns.name = None

pivoted_data_3.head() 

,Area Code,Area,Year Code,Year,Cropland
0,1,Armenia,1992,1992,0.1351
1,1,Armenia,1993,1993,0.1426
2,1,Armenia,1994,1994,0.1482
3,1,Armenia,1995,1995,0.1487
4,1,Armenia,1996,1996,0.1534


In [44]:
# renaming columns
cleaned_data_3 = pivoted_data_3.rename(
    columns = {
        'Area Code': 'area_code',
        'Area': 'area',
        'Year Code': 'year_code',
        'Year': 'year',
        'Cropland': 'cropland_area_per_capita'
    }
)

cleaned_data_3.head()

,area_code,area,year_code,year,cropland_area_per_capita
0,1,Armenia,1992,1992,0.1351
1,1,Armenia,1993,1993,0.1426
2,1,Armenia,1994,1994,0.1482
3,1,Armenia,1995,1995,0.1487
4,1,Armenia,1996,1996,0.1534


In [50]:
cleaned_data_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   area_code                 15098 non-null  int64  
 1   area                      15098 non-null  object 
 2   year_code                 15098 non-null  int64  
 3   year                      15098 non-null  int64  
 4   cropland_area_per_capita  15098 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 589.9+ KB


In [51]:
# merging datasets with left join
merged_data_2 = pd.merge(
    merged_data_1, cleaned_data_3,
    on = ['area_code', 'area', 'year_code', 'year'],
    how = 'left'
)

merged_data_2.head()

,area_code,area,year_code,year,area_agri_land,area_arable_land,area_cropland,area_with_irrigation,area_permanent_crops,area_temporary_crops,value_per_unit_agri_land,cropland_area_per_capita
0,1,Armenia,1992,1992,1170.0,423.0,483.0,276.0,60.0,NaN,NaN,0.1351
1,1,Armenia,1993,1993,1181.0,433.0,493.0,277.8,60.0,NaN,NaN,0.1426
2,1,Armenia,1994,1994,1200.0,435.0,500.0,279.6,65.0,NaN,NaN,0.1482
3,1,Armenia,1995,1995,1244.0,435.0,494.0,281.4,59.0,NaN,NaN,0.1487
4,1,Armenia,1996,1996,1306.0,450.0,506.0,283.2,56.0,NaN,NaN,0.1534


In [52]:
merged_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15354 entries, 0 to 15353
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   area_code                 15354 non-null  int64  
 1   area                      15354 non-null  object 
 2   year_code                 15354 non-null  int64  
 3   year                      15354 non-null  int64  
 4   area_agri_land            15353 non-null  float64
 5   area_arable_land          14974 non-null  float64
 6   area_cropland             15160 non-null  float64
 7   area_with_irrigation      12426 non-null  float64
 8   area_permanent_crops      14123 non-null  float64
 9   area_temporary_crops      5738 non-null   float64
 10  value_per_unit_agri_land  13665 non-null  float64
 11  cropland_area_per_capita  15098 non-null  float64
dtypes: float64(8), int64(3), object(1)
memory usage: 1.4+ MB


In [53]:
# exporting merged and cleaned data as csv file
merged_data_2.to_csv('cleaned_datasets/landuse_cleaned.csv', index='False')